<a href="https://colab.research.google.com/github/gyuminpk/Playground_solver/blob/main/%EC%9C%A1%EA%B0%81%EB%8F%8C%EB%A6%AC%EA%B8%B0_Solver.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [15]:
row_lengths = [3, 4, 5, 4, 3]

def get_adjacencies(row, col, row_lengths):
    adjacencies = []
    num_rows = len(row_lengths)

    def is_valid_coord(r, c):
        return 0 <= r < num_rows and 0 <= c < row_lengths[r]

    # The following logic appends neighbors in a consistent clockwise order:
    # 1. Top-Left
    # 2. Top-Right
    # 3. Right
    # 4. Bottom-Right
    # 5. Bottom-Left
    # 6. Left

    # 1. Top-Left Candidate
    if row > 0:
        prev_row_len = row_lengths[row - 1]
        curr_row_len = row_lengths[row]
        if prev_row_len < curr_row_len: # Previous row is shorter (e.g., row 2 (5) -> row 1 (4))
            tl = (row - 1, col - 1)
        else: # Previous row is longer or equal (e.g., row 3 (4) -> row 2 (5))
            tl = (row - 1, col)
        if is_valid_coord(tl[0], tl[1]):
            adjacencies.append(tl)

    # 2. Top-Right Candidate
    if row > 0:
        prev_row_len = row_lengths[row - 1]
        curr_row_len = row_lengths[row]
        if prev_row_len < curr_row_len:
            ur = (row - 1, col)
        else:
            ur = (row - 1, col + 1)
        if is_valid_coord(ur[0], ur[1]):
            adjacencies.append(ur)

    # 3. Right Candidate
    r_coord = (row, col + 1)
    if is_valid_coord(r_coord[0], r_coord[1]):
        adjacencies.append(r_coord)

    # 4. Bottom-Right Candidate
    if row < num_rows - 1:
        next_row_len = row_lengths[row + 1]
        curr_row_len = row_lengths[row]
        if next_row_len > curr_row_len: # Next row is longer (e.g., row 0 (3) -> row 1 (4))
            if row == 0: # Special case for row 0 with 3 bottom neighbors
                dr = (row + 1, col + 1) # Furthest right bottom neighbor
            else: # For rows > 0 where next row is longer (e.g., row 1 (4) -> row 2 (5))
                dr = (row + 1, col + 1)
        else: # Next row is shorter or equal (e.g., row 2 (5) -> row 3 (4))
            dr = (row + 1, col)
        if is_valid_coord(dr[0], dr[1]):
            adjacencies.append(dr)

    # 5. Bottom-Left Candidate
    if row < num_rows - 1:
        next_row_len = row_lengths[row + 1]
        curr_row_len = row_lengths[row]
        if next_row_len > curr_row_len:
            if row == 0: # Special case for row 0 with 3 bottom neighbors
                dl = (row + 1, col) # Middle bottom neighbor
            else:
                dl = (row + 1, col)
        else:
            dl = (row + 1, col - 1)
        if is_valid_coord(dl[0], dl[1]):
            adjacencies.append(dl)

    # 6. Left Candidate
    l_coord = (row, col - 1)
    if is_valid_coord(l_coord[0], l_coord[1]):
        adjacencies.append(l_coord)

    # Remove duplicates while preserving order
    final_adj = []
    seen = set()
    for coord in adjacencies:
        if coord not in seen:
            final_adj.append(coord)
            seen.add(coord)
    return final_adj


# print("Board row lengths defined: " + str(row_lengths))
# print("Testing get_adjacencies for (0, 1):", get_adjacencies(0, 1, row_lengths))
# print("Testing get_adjacencies for (1, 1):", get_adjacencies(1, 1, row_lengths))
# print("Testing get_adjacencies for (2, 2) (center):", get_adjacencies(2, 2, row_lengths))
# print("Testing get_adjacencies for (4, 1):", get_adjacencies(4, 1, row_lengths))

rotatable_marbles = [
    (1, 1), (1, 2),  # Row 2 (index 1): 2nd and 3rd marbles
    (2, 1), (2, 2), (2, 3),  # Row 3 (index 2): 2nd, 3rd, and 4th marbles
    (3, 1), (3, 2)   # Row 4 (index 3): 2nd and 3rd marbles
]

def parse_board_state(state_string, row_lengths):
    rows_str = state_string.split('/')
    board_state = []

    if len(rows_str) != len(row_lengths):
        raise ValueError(f"Expected {len(row_lengths)} rows, but got {len(rows_str)}")

    for i, row_str in enumerate(rows_str):
        if len(row_str) != row_lengths[i]:
            raise ValueError(f"Row {i} has length {len(row_str)}, but expected {row_lengths[i]}")

        # Convert each character in the row string to an integer and store as a tuple
        board_state.append(tuple(int(marble) for marble in row_str))

    return tuple(board_state)

def board_state_to_string(board_state):
    # Convert each tuple of integers back to a string of digits
    row_strings = []
    for row_tuple in board_state:
        row_strings.append("".join(str(marble) for marble in row_tuple))

    # Join the row strings with '/' to return the original format
    return "/".join(row_strings)
from collections import defaultdict

def validate_marble_counts(board_state):
    expected_counts = {1: 6, 2: 6, 3: 6, 4: 1}
    actual_counts = defaultdict(int)

    # Iterate through the board state to count marbles
    for row_tuple in board_state:
        for marble in row_tuple:
            actual_counts[marble] += 1

    # Check if actual counts match expected counts
    if actual_counts.keys() != expected_counts.keys():
        print(f"Marble types mismatch. Expected: {sorted(expected_counts.keys())}, Actual: {sorted(actual_counts.keys())}")
        return False

    for marble_type, expected_count in expected_counts.items():
        if actual_counts[marble_type] != expected_count:
            print(f"Marble type {marble_type}: Expected {expected_count}, Found {actual_counts[marble_type]}")
            return False

    print("Marble counts are valid.")
    return True

def apply_rotation(current_state, central_marble_coords, row_lengths):
    # 1. Create a mutable copy of the current_state (tuple of tuples -> list of lists)
    new_board_list = [list(row) for row in current_state]

    # 2. Get the coordinates of the adjacent marbles
    adj_coords = get_adjacencies(central_marble_coords[0], central_marble_coords[1], row_lengths)

    # 3. Extract the values of the marbles at these adjacent coordinates
    #    Preserve order for rotation.
    adj_marble_values = [new_board_list[r][c] for r, c in adj_coords]

    # 4. Implement a clockwise shift on these extracted marble values
    #    The last element moves to the first position, others shift right.
    if adj_marble_values:
        shifted_values = [adj_marble_values[-1]] + adj_marble_values[:-1]
    else:
        shifted_values = []

    # 5. Update the mutable board copy by placing the shifted marble values back
    for i, (r, c) in enumerate(adj_coords):
        new_board_list[r][c] = shifted_values[i]

    # 6. Convert the modified mutable board copy back into a tuple of tuples
    return tuple(tuple(row) for row in new_board_list)


from collections import deque

def fast_heuristic(state, goal):
    h = 0
    rows = len(state)

    for r in range(rows):
        for c in range(len(state[r])):
            if state[r][c] != goal[r][c]:
                # 위쪽일수록 훨씬 큰 패널티
                h += (rows - r) * 3
    return h

import heapq

def solve_marble_puzzle_greedy(initial_state, goal_state, rotatable_marbles, row_lengths):
    # (h, state, path, last_move)
    heap = []
    heapq.heappush(heap, (fast_heuristic(initial_state, goal_state),
                           initial_state, [], None))

    visited = set()

    while heap:
        h, state, path, last_move = heapq.heappop(heap)

        if state == goal_state:
            return path

        if state in visited:
            continue
        visited.add(state)

        for center in rotatable_marbles:
            # 🔥 pruning 1: 같은 회전 연속 금지
            if center == last_move:
                continue

            next_state = apply_rotation(state, center, row_lengths)

            # 🔥 pruning 2: 변화 없으면 스킵
            if next_state == state:
                continue

            heapq.heappush(
                heap,
                (fast_heuristic(next_state, goal_state),
                 next_state,
                 path + [center],
                 center)
            )

    return None
import time
import random

def solve_marble_puzzle_multi_greedy(
    initial_state,
    goal_state,
    rotatable_marbles,
    row_lengths,
    max_runs=200,
    time_limit=5.0
):
    best_solution = None
    start_time = time.time()

    for run in range(max_runs):
        if time.time() - start_time > time_limit:
            break

        # 🔀 회전 순서 랜덤화
        moves = rotatable_marbles[:]
        random.shuffle(moves)

        solution = solve_marble_puzzle_greedy(
            initial_state,
            goal_state,
            moves,
            row_lengths
        )

        if solution:

            if best_solution is None or len(solution) < len(best_solution):
                best_solution = solution
                print(f"🔥 New best: {len(best_solution)} moves (run {run})")

                # 충분히 짧으면 조기 종료
                if len(best_solution) <= 40:
                    break

    return best_solution




In [16]:
import sys

# 1. Define the initial and goal board states as strings
initial_board_state_string = '223/3341/23122/1133/121'
goal_board_state_string = '111/1112/33422/3322/332'

print(f"Initial board state string: {initial_board_state_string}")
print(f"Goal board state string: {goal_board_state_string}\n")

# 2. Parse both states into their internal tuple-of-tuples representations
try:
    initial_board_state = parse_board_state(initial_board_state_string, row_lengths)
    goal_board_state = parse_board_state(goal_board_state_string, row_lengths)
except ValueError as e:
    print(f"Error parsing board state: {e}")
    sys.exit(1)

print(f"Parsed initial board state: {initial_board_state}")
print(f"Parsed goal board state: {goal_board_state}\n")

# 3. Validate the initial_board_state
print("Validating initial board state marble counts...")
if not validate_marble_counts(initial_board_state):
    print("Initial board state has invalid marble counts. Aborting.")
    sys.exit(1)
print("Initial board state marble counts are valid. Proceeding with Greedy.\n")

# 4. Call the solve_marble_puzzle_astar function
print("Searching for a solution using Greedy...")
solution_path = solve_marble_puzzle_multi_greedy(
    initial_board_state,
    goal_board_state,
    rotatable_marbles,
    row_lengths,
    max_runs=300,
    time_limit=8.0
)

print("Best solution:", solution_path)
print("Moves:", len(solution_path))



Initial board state string: 223/3341/23122/1133/121
Goal board state string: 111/1112/33422/3322/332

Parsed initial board state: ((2, 2, 3), (3, 3, 4, 1), (2, 3, 1, 2, 2), (1, 1, 3, 3), (1, 2, 1))
Parsed goal board state: ((1, 1, 1), (1, 1, 1, 2), (3, 3, 4, 2, 2), (3, 3, 2, 2), (3, 3, 2))

Validating initial board state marble counts...
Marble counts are valid.
Initial board state marble counts are valid. Proceeding with Greedy.

Searching for a solution using Greedy...
🔥 New best: 125 moves (run 0)
Best solution: [(2, 3), (2, 1), (3, 1), (2, 1), (1, 1), (2, 3), (2, 1), (3, 2), (1, 1), (1, 2), (2, 3), (3, 1), (2, 3), (3, 2), (3, 1), (2, 2), (2, 1), (3, 2), (2, 1), (3, 1), (3, 2), (3, 1), (2, 2), (3, 2), (3, 1), (3, 2), (3, 1), (2, 2), (3, 1), (3, 2), (3, 1), (3, 2), (3, 1), (3, 2), (3, 1), (2, 2), (3, 2), (2, 3), (3, 2), (3, 1), (2, 2), (3, 1), (3, 2), (3, 1), (3, 2), (3, 1), (2, 2), (3, 2), (3, 1), (3, 2), (3, 1), (3, 2), (3, 1), (3, 2), (3, 1), (3, 2), (3, 1), (2, 2), (3, 2), (3, 1)